In [1]:
#Snowpark lib
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

#ConfigParser to read ini file
import configparser

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import DBSCAN

np.random.seed(0)

config = configparser.ConfigParser()
config.read("/notebooks/notebooks/credentials.ini")

connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [2]:
# df = session.table("MEMBER_FUNDS_ENRICHED_DETAILS").to_pandas()
df = pd.read_csv("/data/funds.csv")

In [3]:
# df.to_csv("/data/funds.csv", index=False)

In [3]:
df.head()

,MEMBER_ID,MEMBER_NAME,MEMBER_EMPLOYMENT,MEMBER_GENDER,MEMBER_CITY_TOWN,MEMBER_STATE,MEMBER_CONTACT_VERIFIED,FUND_ID,CHURN_REASON,LATITUDE,...,MEMBER_AGE,INVESTMENT_AGE_GROUP,RETIREMENT_AGE,RETIREMENT_AGE_GROUP,TOTAL_FUNDS_INVESTED,NO_OF_CHURN,MEMBER_DOB_DT,ALLOCATION_DT,CHURN_DT,RETIREMENT_DT
0,MID108406,Maile Treston,Charles Darwin University,Male,Lakes Entrance,Victoria,Y,FID000044,NOT APPLICABLE,-37.8667,...,52,Senior,13,MORE_THAN_10_YEAR,3,2.0,1972-04-29,2011-02-25,9999-12-31,2037-04-29
1,MID108415,Annabelle Cryer,RMIT University,Female,Mooroopna,Victoria,N,FID000016,NOT APPLICABLE,-36.3833,...,26,Young,39,MORE_THAN_10_YEAR,3,2.0,1998-10-05,2010-01-04,9999-12-31,2063-10-05
2,MID108429,Leonora Acuff,University of South Australia,Male,Jindabyne,New South Wales,N,FID000100,NOT APPLICABLE,-36.4000,...,51,Senior,14,MORE_THAN_10_YEAR,3,2.0,1973-12-20,2010-09-04,9999-12-31,2038-12-20
3,MID108436,Franklyn Gesick,The University of Newcastle,Male,Wingham,New South Wales,N,FID000012,NOT APPLICABLE,-31.8667,...,41,Middle-aged,24,MORE_THAN_10_YEAR,3,2.0,1983-03-14,2013-04-27,9999-12-31,2048-03-14
4,MID108438,Amber Gillaspie,University of South Australia,Female,Nathalia,Victoria,Y,FID000035,NOT APPLICABLE,-36.0583,...,40,Middle-aged,25,MORE_THAN_10_YEAR,3,2.0,1984-01-06,2012-05-17,9999-12-31,2049-01-06


In [4]:
df.isna().sum() / df.shape[0]

MEMBER_ID                                           0.000000
MEMBER_NAME                                         0.000000
MEMBER_EMPLOYMENT                                   0.000000
MEMBER_GENDER                                       0.000000
MEMBER_CITY_TOWN                                    0.000000
MEMBER_STATE                                        0.000000
MEMBER_CONTACT_VERIFIED                             0.000000
FUND_ID                                             0.000000
CHURN_REASON                                        0.000000
LATITUDE                                            0.000000
LONGITUDE                                           0.000000
COUNTRY                                             0.000000
COUNTRYCODE                                         0.000000
CAPITAL                                             0.000000
POPULATION                                          0.000000
FUND_TOTAL_ASSETS                                   0.000000
FUND_RETURN_TARGET_PERCE

NO_OF_CHURN column has almost 50% missing value, it wont be wise to impute these many values so will just drop it.                                     

In [5]:
frame = df[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED','CHURN_FLAG']].copy()

In [6]:
frame["CHURN_FLAG"] = frame["CHURN_FLAG"].apply(lambda x: 1 if x =="Y" else 0)

In [82]:
# frame.groupby(["RETIREMENT_AGE_GROUP","CHURN_FLAG"])[["CHURN_FLAG"]].count()

The distribution of CHURN is pretty common bettween the employers and hence wont contribute inn modelling

In [76]:
# multiple = {i for i, j in dict(df["MEMBER_ID"].value_counts()).items() if j > 1 }
# multiple[multiple["value_counts"]>=2].index.to_list()

# MODEL TO PREDICT CHURN

In [7]:
numeric_features = ["FUND_TOTAL_ASSETS", 'FUND_RETURN_TARGET_PERCENTAGE','CASH_BENCHMARK_ALLOCATION',
       'FIXED_INCOME_BENCHMARK_ALLOCATION','DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
       'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION', 'INFRA_BENCHMARK_ALLOCATION',
       'COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS',
       'YEAR_5_RETURNS', 'YEAR_7_RETURNS', 'YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ["MEMBER_GENDER", "MEMBER_STATE", "MEMBER_CONTACT_VERIFIED","INVESTMENT_RISK_CATEGORY",'FUND_RISK_LEVEL',
                        'FUND_RISK_CATEGORY',"NEG_NETRETURN_SINCE_INCEPTION", 'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', "TOTAL_FUNDS_INVESTED"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [8]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier(n_estimators=500, max_depth=8, 
                                                                                 max_features=0.6,
                                                                                 bootstrap=True, max_samples=0.8))]
)



In [9]:
X = frame.drop("CHURN_FLAG", axis=1)
y = frame["CHURN_FLAG"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [10]:
clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.984


In [11]:
import pickle

# save
with open('churn_model.pkl','wb') as f:  ## use rb while reading the fie
    pickle.dump(clf,f)

In [12]:
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['FUND_TOTAL_ASSETS',
                                                   'FUND_RETURN_TARGET_PERCENTAGE',
                                                   'CASH_BENCHMARK_ALLOCATION',
                                                   'FIXED_INCOME_BENCHMARK_ALLOCATION',
                                                   'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION',
                                                   'INTERNATIONAL_LISTED_EQU...
                                                                                    score_func=<function chi2 at 0x7fbe02958dc0>))]),
                                                  ['MEMBER_GENDER',
                                                   'MEMBER_STATE',
                                                   'MEMBER_CONTACT_VERIFIED',
                                                   'INVESTMENT_RISK_CATEGORY',
                                                   'FUND_RISK_LEVEL',
                                                   'FUND_RISK_CATEGORY',
                                                   'NEG_NETRETURN_SINCE_INCEPTION',
                                                   'INVESTMENT_AGE_GROUP',
                                                   'RETIREMENT_AGE_GROUP',
                                                   'TOTAL_FUNDS_INVESTED'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=8, max_features=0.6,
                                        max_samples=0.8, n_estimators=500))])

In [14]:
frame = clf.predict(frame)
df["CHURN_PREDICTED"] = frame

# MODEL_REASON

In [13]:
churned = df[df["CHURN_FLAG"] == "Y"]

In [14]:
churned.groupby(["CHURN_REASON"])[["CHURN_REASON"]].count() / churned.shape[0]*100

,CHURN_REASON
CHURN_REASON,
FUND REPUTATION DECLINING,29.070673
HIGH ACCOUNT FEE,16.143163
HIGH TRANSACTION FEE,21.015857
LIFE EVENT,5.547624
NO REASON IDENTIFIED,25.771735
POOR FUND PERFORMANCE,2.450949


In [15]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [16]:
frame = churned[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED',"CHURN_REASON"]].copy()

In [17]:
churn_reason_dic = {reason:i for i, reason in enumerate(churned["CHURN_REASON"].unique())}
frame["CHURN_REASON"] = frame["CHURN_REASON"].apply(lambda x: churn_reason_dic[x])

In [18]:
numeric_features = ["FUND_TOTAL_ASSETS", 'FUND_RETURN_TARGET_PERCENTAGE','CASH_BENCHMARK_ALLOCATION',
       'FIXED_INCOME_BENCHMARK_ALLOCATION','DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
       'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION', 'INFRA_BENCHMARK_ALLOCATION',
       'COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS',
       'YEAR_5_RETURNS', 'YEAR_7_RETURNS', 'YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ["MEMBER_GENDER", "MEMBER_STATE", "MEMBER_CONTACT_VERIFIED","INVESTMENT_RISK_CATEGORY",'FUND_RISK_LEVEL',
                        'FUND_RISK_CATEGORY',"NEG_NETRETURN_SINCE_INCEPTION", 'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', "TOTAL_FUNDS_INVESTED"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [19]:
multi_lable_clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=500, max_depth=8, max_features=0.6, bootstrap=True,
                                                             max_samples=0.8))
clf2 = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", multi_lable_clf)]
)


X = frame.drop("CHURN_REASON", axis=1)
y = frame["CHURN_REASON"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape)

clf2.fit(X_train, y_train)
print("model score: %.3f" % clf2.score(X_test, y_test))

(73963, 29)
model score: 1.000


In [20]:
import pickle

# save
with open('reason_model.pkl','wb') as f:  ## use rb while reading the fie
    pickle.dump(clf2,f)

# FUND RECOMMENDATION

In [39]:
fund_allocation = session.table("FUND_ALLOCATION_MASTER").to_pandas()
fund_master = session.table("FUND_MASTER").to_pandas()
fund_performance = session.table("FUND_PERFORMANCE_MASTER").to_pandas()

In [40]:
fund = pd.merge(fund_allocation, fund_master, on="FUND_ID")
fund_total = pd.merge(fund, fund_performance, on="FUND_ID")
fund_charge = [float(i) for i in fund_total["FUND_FEES_CHARGES"] if i != 'Not Available']
avg_fund_charge = sum(fund_charge) / len(fund_charge)

fund_total["FUND_FEES_CHARGES"] = fund_total["FUND_FEES_CHARGES"].apply(lambda x: avg_fund_charge if x == 'Not Available' else x)
fund_total["FUND_FEES_CHARGES"] = fund_total["FUND_FEES_CHARGES"].astype(float) 

fund_meta = fund_total[["FUND_ID","FUNDNAME"]]
fund_cluster = fund_total.drop(["FUND_ID","FUNDNAME","FUND_NAME_y","FUND_TRUSTEE","FUND_INCEPTION_DATE"], axis=1)

In [41]:
fund_data = pd.get_dummies(fund_cluster)

fund_data = fund_data.replace(False, 0)
fund_data = fund_data.replace(True, 1)

In [42]:
clustering = DBSCAN(eps=2, min_samples=3).fit(fund_data)

In [43]:
fund_total["cluster"] = clustering.labels_
fund_total["avg_return"] = fund_total[["YEAR_1_RETURNS","YEAR_3_RETURNS",
                                 "YEAR_5_RETURNS","YEAR_7_RETURNS",
                                 "YEAR_10_RETURNS"]].apply(lambda x:
                                                        (x[0] + (0.9*x[1]) + (0.75*x[2]) + (0.6*x[3]) + (0.55*x[4]))/5, axis=1)

In [54]:
def recommendation(fund_id, reason):
    if reason == 'NO REASON IDENTIFIED' or reason == "NOT APPLICABLE":
        return None
    elif reason == 'HIGH ACCOUNT FEE' or reason == 'HIGH TRANSACTION FEE':
            clus_frame = fund_total[fund_total["FUND_ID"] == fund_id]
            clus = list(clus_frame["cluster"])[0]
            clus_frame["charge"] = clus_frame[["SUPER_FEES","PENSION_FEES","FUND_FEES_CHARGES"]].apply(lambda x: (x[0]+x[1]+x[2])/3, axis=1)
            total_charge = list(clus_frame["charge"])[0]
            group = fund_total[fund_total["cluster"] == clus]
            group["avg_fee"] = group[["SUPER_FEES","PENSION_FEES","FUND_FEES_CHARGES"]].apply(lambda x: (x[0]+x[1]+x[2])/3, axis=1)
            group["rec"] = group["avg_fee"].apply(lambda x: 1 if x < total_charge else 0)
            group = group.sort_values("avg_return")
            recommendations = list(group[group["rec"] == 1]["FUND_ID"])
            if fund_id in recommendations:
                recommendations.remove(fund_id)
            recommendations = recommendations[:5]
            return recommendations
#             return group
    elif reason == 'FUND REPUTATION DECLINING' or reason == 'POOR FUND PERFORMANCE':
        
        pass
        
            
            
            
rec = recommendation("FID000014","HIGH ACCOUNT FEE")
print(len(rec))

1


In [55]:
rec

['FID000016']

In [115]:
['FID000103', 'FID000067', 'FID000080', 'FID000097', 'FID000186']
['FID000103', 'FID000067', 'FID000080', 'FID000142', 'FID000097']

array(['NO REASON IDENTIFIED', 'HIGH ACCOUNT FEE', 'HIGH TRANSACTION FEE',
       'FUND REPUTATION DECLINING', 'LIFE EVENT', 'POOR FUND PERFORMANCE'],
      dtype=object)

In [59]:
temp_ = list(churned[churned["CHURN_REASON"] == "FUND REPUTATION DECLINING"]["FUND_ID"])
fund_total[fund_total["FUND_ID"].apply(lambda x: x in temp_)]

,FUND_ID,FUNDNAME,FUND_TOTAL_ASSETS,FUND_RETURN_TARGET_PERCENTAGE,INVESTMENT_RISK_LEVEL,INVESTMENT_RISK_CATEGORY,CASH_BENCHMARK_ALLOCATION,FIXED_INCOME_BENCHMARK_ALLOCATION,DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION,INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION,...,NEG_NETRETURN_SINCE_INCEPTION,YEAR_1_RETURNS,YEAR_3_RETURNS,YEAR_5_RETURNS,YEAR_7_RETURNS,YEAR_10_RETURNS,SUPER_FEES,PENSION_FEES,cluster,avg_return
0,FID000001,AMG Super,2115.4,2.765926,4.000000,Medium-High,5.000000,25.000000,35.000000,25.000000,...,2 to 3,6.03,4.89,4.69,5.04,5.32,0.50,0.44,-1,3.9797
1,FID000002,AMG Super,2115.4,2.765926,4.000000,Medium-High,5.000000,25.000000,35.000000,25.000000,...,2 to 3,6.18,4.18,4.42,5.15,5.44,0.50,0.40,-1,3.8678
2,FID000003,AMP Retirement Trust,15660.6,4.088889,3.907037,Medium-High,5.000000,25.000000,27.000000,31.000000,...,2 to 3,6.05,4.05,4.85,5.22,5.55,0.53,0.45,-1,3.9034
3,FID000004,AMP Retirement Trust,15660.6,4.088889,3.907037,Medium-High,5.000000,25.000000,27.000000,31.000000,...,2 to 3,6.14,4.60,4.66,5.19,5.50,0.50,0.40,-1,3.9828
6,FID000007,AMP Superannuation Savings Trust,313827.1,3.879487,3.394615,Medium-Low,5.000000,25.000000,25.000000,26.000000,...,1 to 2,6.44,4.89,4.17,5.01,5.43,0.52,0.48,0,3.9922
8,FID000009,AMP Superannuation Savings Trust,313827.1,3.879487,3.394615,Medium-Low,5.000000,25.000000,25.000000,26.000000,...,1 to 2,6.48,4.79,4.76,5.21,5.54,0.50,0.49,0,4.1068
9,FID000010,ANZ Australian Staff Superannuation Scheme,46960.4,3.985185,4.188889,Medium-High,3.000000,17.000000,20.000000,30.000000,...,2 to 3,6.12,4.21,4.68,5.02,5.36,0.52,0.45,-1,3.8758
18,FID000019,Australian Ethical Retail Superannuation Fund,17218.1,3.500000,3.203571,Medium-Low,5.000000,24.000000,29.000000,21.500000,...,1 to 2,6.38,4.62,4.07,5.15,5.36,0.52,0.50,-1,3.9257
21,FID000022,Australian Retirement Trust,921618.2,2.142857,3.457143,Medium-Low,10.000000,10.000000,10.000000,10.000000,...,1 to 2,6.28,4.22,4.03,5.05,5.55,0.50,0.49,-1,3.8366
24,FID000025,Austsafe Superannuation Fund,37070.7,4.453636,4.009091,Medium-High,5.000000,13.000000,27.000000,27.000000,...,2 to 3,6.16,4.29,4.54,5.14,5.62,0.53,0.45,-1,3.9202


In [ ]:
y = mx+c